In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd

asv_df = pd.read_csv("/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Dataset/asv_key.csv")
thai_df = pd.read_csv("/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Dataset/thai_key.csv")


In [ ]:
len(asv_df)

56251

In [ ]:
eval = asv_df[asv_df["7"] == 'eval'] # test
hidden = asv_df[asv_df["7"] == 'hidden'] # eval
progress = asv_df[asv_df["7"] == 'progress'] # train

thai_eval = thai_df[thai_df["set"] == "eval"] #thai train
thai_train = thai_df[thai_df["set"] == "train"] #thai eval

In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset

def stack_loader(X, y):
  X_tensor = torch.tensor(X, dtype=torch.float32)
  y_tensor = torch.tensor(y)
  dataset = TensorDataset(X_tensor, y_tensor)
  data_loader = DataLoader(dataset, batch_size=16, shuffle=True)
  return data_loader

# **Eng**

In [ ]:
# Wav2vec
import numpy as np

progress_Wav2vec = np.load("/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Features/4sec/Wav2vec/Wav2vec_progress.npy", allow_pickle=True)
hidden_Wav2vec = np.load("/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Features/4sec/Wav2vec/Wav2vec_hidden.npy", allow_pickle=True)
# eval_Wav2vec = np.load("/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Features/4sec/Wav2vec/Wav2vec_eval.npy")



In [ ]:
progress_Wav2vec.shape

(5055, 64000)

In [ ]:
#Label
import numpy as np
from sklearn.model_selection import train_test_split

X_train_val = np.vstack([progress_Wav2vec, hidden_Wav2vec])
y_train = np.hstack([progress["labels"], hidden["labels"]])

# X_test = np.vstack([eval_VGGish])
# y_test = np.hstack([eval["labels"]])
# Split the data into training and validation sets

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train, test_size=0.2, random_state=42)


In [ ]:
del progress_Wav2vec
del hidden_Wav2vec

del X_train_val

In [ ]:
print("Shape of X_train:", X_train.shape) # progress 5055, hidden 5244
print("Shape of X_val:", X_val.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_val:", y_val.shape)

Shape of X_train: (8239, 64000)
Shape of X_val: (2060, 64000)
Shape of y_train: (8239,)
Shape of y_val: (2060,)


In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset

train_loader = stack_loader(X_train, y_train)
val_loader = stack_loader(X_val, y_val)

# **Thai**

In [ ]:
!ls "/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Features/4sec/Wav2vec"

Wav2vec_eval.npy    Wav2vec_progress.npy   Wav2vec_thai_train.npy
Wav2vec_hidden.npy  Wav2vec_thai_eval.npy


In [ ]:
import numpy as np
# Thai

thai_Wav2vec = np.load("/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Features/4sec/Wav2vec/Wav2vec_thai_train.npy", allow_pickle=True)

In [ ]:
thai_Wav2vec.shape

(12000, 64000)

In [ ]:
#Thai
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(thai_Wav2vec, thai_train["labels"], test_size=0.2, random_state=42)

import torch
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset

def stack_loader(X, y):
  X_tensor = torch.tensor(X, dtype=torch.float32)
  y_tensor = torch.tensor(y.values)
  dataset = TensorDataset(X_tensor, y_tensor)
  data_loader = DataLoader(dataset, batch_size=16, shuffle=True)
  return data_loader

train_loader = stack_loader(X_train, y_train)
val_loader = stack_loader(X_val, y_val)

In [ ]:
print("Shape of X_train:", X_train.shape) # thai real 1000, thai fake 11000
print("Shape of X_val:", X_val.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_val:", y_val.shape)

Shape of X_train: (9600, 64000)
Shape of X_val: (2400, 64000)
Shape of y_train: (9600,)
Shape of y_val: (2400,)


# **Thai+Eng**

In [ ]:
import numpy as np

progress_Wav2vec = np.load("/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Features/4sec/Wav2vec/Wav2vec_progress.npy", allow_pickle=True)
hidden_Wav2vec = np.load("/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Features/4sec/Wav2vec/Wav2vec_hidden.npy", allow_pickle=True)
thai_Wav2vec = np.load("/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Features/4sec/Wav2vec/Wav2vec_thai_train.npy", allow_pickle=True)


X_train_val = np.vstack([thai_Wav2vec, progress_Wav2vec, hidden_Wav2vec])
y_train = np.hstack([thai_train["labels"], progress["labels"], hidden["labels"]])

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train, test_size=0.2, random_state=42)

In [ ]:
print("Shape of X_train:", X_train.shape) # thai real 1000, thai fake 11000, progress 5055, hidden 5244
print("Shape of X_val:", X_val.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_val:", y_val.shape)

Shape of X_train: (17839, 64000)
Shape of X_val: (4460, 64000)
Shape of y_train: (17839,)
Shape of y_val: (4460,)


In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset

def stack_loader(X, y):
  X_tensor = torch.tensor(X, dtype=torch.float32)
  y_tensor = torch.tensor(y)
  dataset = TensorDataset(X_tensor, y_tensor)
  data_loader = DataLoader(dataset, batch_size=16, shuffle=True)
  return data_loader

train_loader = stack_loader(X_train, y_train)
val_loader = stack_loader(X_val, y_val)

# **RawNet2**

In [ ]:
!ls "/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Model"

AST  LFCC  Mel	Raw  Wav2vec


In [ ]:
!git clone "https://github.com/asvspoof-challenge/2021.git"

Cloning into '2021'...
remote: Enumerating objects: 483, done.
remote: Counting objects: 100% (483/483), done.
remote: Compressing objects: 100% (309/309), done.
remote: Total 483 (delta 167), reused 450 (delta 135), pack-reused 0
Receiving objects: 100% (483/483), 2.30 MiB | 19.33 MiB/s, done.
Resolving deltas: 100% (167/167), done.


In [ ]:
%cd "/content/2021/LA/Baseline-RawNet2"

/content/2021/LA/Baseline-RawNet2


In [ ]:
import argparse
import sys
import os
import numpy as np
import torch
from torch import nn
from torch import Tensor
from torch.utils.data import DataLoader
import yaml
from data_utils import genSpoof_list,Dataset_ASVspoof2019_train,Dataset_ASVspoof2021_eval
from model import RawNet
# from tensorboardX import SummaryWriter
from core_scripts.startup_config import set_random_seed
from sklearn.metrics import roc_curve
from scipy.optimize import brentq
from scipy.interpolate import interp1d

def evaluate_eer(dev_loader, model, device):
    model.eval()
    all_scores = []
    all_labels = []
    for batch_x, batch_y in dev_loader:
        batch_x = batch_x.to(device)
        batch_out = model(batch_x)
        scores = torch.softmax(batch_out, dim=1)[:, 1].cpu().detach().numpy()  # Get the probabilities for class 1
        labels = batch_y.cpu().detach().numpy()
        all_scores.extend(scores)
        all_labels.extend(labels)

    fpr, tpr, thresholds = roc_curve(all_labels, all_scores, pos_label=1)
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    return eer

def evaluate_accuracy(dev_loader, model, device):
    num_correct = 0.0
    num_total = 0.0
    model.eval()
    for batch_x, batch_y in dev_loader:

        batch_size = batch_x.size(0)
        num_total += batch_size
        batch_x = batch_x.to(device)
        batch_y = batch_y.view(-1).type(torch.int64).to(device)
        batch_out = model(batch_x)
        _, batch_pred = batch_out.max(dim=1)
        num_correct += (batch_pred == batch_y).sum(dim=0).item()
    return 100 * (num_correct / num_total)

def train_epoch(train_loader, model, lr,optim, device):
    running_loss = 0
    num_correct = 0.0
    num_total = 0.0
    ii = 0
    model.train()

    #set objective (Loss) functions
    weight = torch.FloatTensor([0.1, 0.9]).to(device)
    criterion = nn.CrossEntropyLoss(weight=weight)

    for batch_x, batch_y in train_loader:

        batch_size = batch_x.size(0)
        num_total += batch_size
        ii += 1
        batch_x = batch_x.to(device)
        batch_y = batch_y.view(-1).type(torch.int64).to(device)
        batch_out = model(batch_x)
        batch_loss = criterion(batch_out, batch_y)
        _, batch_pred = batch_out.max(dim=1)
        num_correct += (batch_pred == batch_y).sum(dim=0).item()
        running_loss += (batch_loss.item() * batch_size)
        if ii % 10 == 0:
            sys.stdout.write('\r \t {:.2f}'.format(
                (num_correct/num_total)*100))
        optim.zero_grad()
        batch_loss.backward()
        optim.step()

    running_loss /= num_total
    train_accuracy = (num_correct/num_total)*100
    return running_loss, train_accuracy

In [ ]:
if __name__ == '__main__':

    model_save_path = "/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Model"
    dir_yaml = os.path.splitext('model_config_RawNet')[0] + '.yaml'

    with open(dir_yaml, 'r') as f_yaml:
        parser1 = yaml.load(f_yaml, Loader=yaml.Loader)

    #GPU device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print('Device: {}'.format(device))

    #make experiment reproducible
    set_random_seed(1234)

    #model
    model = RawNet(parser1['model'], device)
    nb_params = sum([param.view(-1).size()[0] for param in model.parameters()])
    model =(model).to(device)

Device: cpu


In [ ]:
if __name__ == '__main__':

    model_save_path = "/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Model"
    dir_yaml = os.path.splitext('model_config_RawNet')[0] + '.yaml'

    with open(dir_yaml, 'r') as f_yaml:
        parser1 = yaml.load(f_yaml, Loader=yaml.Loader)

    #GPU device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print('Device: {}'.format(device))

    #make experiment reproducible
    set_random_seed(1234)

    #model
    model = RawNet(parser1['model'], device)
    nb_params = sum([param.view(-1).size()[0] for param in model.parameters()])
    model =(model).to(device)
    #set Adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001,weight_decay=0.0001)

    model_format = 'RawNet2_Wav2vec_Thai_Eng_epoch_{}.pth'
    num_epochs = 100
    best_acc = 99
    for epoch in range(num_epochs):
        running_loss, train_accuracy = train_epoch(train_loader,model, 0.0001, optimizer, device)
        valid_accuracy = evaluate_accuracy(val_loader, model, device)
        print('\n{} - {} - {:.2f} - {:.2f}'.format(epoch,
                                                   running_loss, train_accuracy, valid_accuracy))
        if valid_accuracy > best_acc:
            print('best model find at epoch', epoch)

        if epoch % 5 == 0 :
            best_acc = max(valid_accuracy, best_acc)
            torch.save(model.state_dict(), os.path.join(model_save_path, model_format.format(epoch)))

    torch.save(model.state_dict(), os.path.join(model_save_path, model_format.format(epoch)))

Device: cuda
 	 78.83
0 - 0.6418081384294804 - 78.78 - 70.72
 	 77.33
1 - 0.5998637737951991 - 77.33 - 80.65
 	 78.99
2 - 0.5757080679671153 - 78.97 - 82.44
 	 79.80
3 - 0.5610035373708968 - 79.78 - 77.98
 	 79.65
4 - 0.5477754449478827 - 79.63 - 76.59
 	 80.72
5 - 0.5126932127919858 - 80.71 - 79.42
 	 80.73
6 - 0.4823536986767146 - 80.75 - 82.85
 	 80.83
7 - 0.45386617006699037 - 80.84 - 78.97
 	 82.08
8 - 0.4346293568723077 - 82.10 - 85.25
 	 82.56
9 - 0.3958395758470523 - 82.56 - 83.21
 	 83.72
10 - 0.37440036608035593 - 83.72 - 84.91
 	 85.16
11 - 0.34573742991886675 - 85.16 - 83.97
 	 86.30
12 - 0.3216895668474145 - 86.31 - 81.70
 	 86.78
13 - 0.30237065360974996 - 86.82 - 87.17
 	 87.79
14 - 0.2799186656257084 - 87.79 - 86.26
 	 88.96
15 - 0.24968452118301654 - 88.97 - 85.02
 	 89.55
16 - 0.2411299396960536 - 89.53 - 89.55
 	 90.71
17 - 0.21917053684487833 - 90.71 - 82.58
 	 91.67
18 - 0.2079746029121244 - 91.66 - 82.76
 	 91.65
19 - 0.19148146232348986 - 91.67 - 89.55
 	 92.36
2

In [ ]:
#eng 55
95
99

# **Test Model**

In [ ]:
import os
import numpy as np
import torch
from model import RawNet
from torch.utils.data import DataLoader, TensorDataset

# Assuming thaireal_Wav2vec and thaifake_Wav2vec are defined elsewhere

model_save_path = "/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Model"
dir_yaml = os.path.splitext('model_config_RawNet')[0] + '.yaml'

# Load model configuration from YAML file
with open(dir_yaml, 'r') as f_yaml:
    parser1 = yaml.load(f_yaml, Loader=yaml.Loader)

# GPU device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: {}'.format(device))

model_path = os.path.join(model_save_path, 'RawNet2_Wav2vec_Thai_Eng_epoch_99.pth')

# Create an instance of the model
model = RawNet(parser1['model'], device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()


Device: cpu


RawNet(
  (Sinc_conv): SincConv()
  (first_bn): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (selu): SELU(inplace=True)
  (block0): Sequential(
    (0): Residual_block(
      (lrelu): LeakyReLU(negative_slope=0.3)
      (conv1): Conv1d(20, 20, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(20, 20, kernel_size=(3,), stride=(1,), padding=(1,))
      (mp): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (block1): Sequential(
    (0): Residual_block(
      (bn1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (lrelu): LeakyReLU(negative_slope=0.3)
      (conv1): Conv1d(20, 20, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(20, 20, kernel_size=(3,), stride

In [ ]:
eval_Wav2vec = np.load("/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Features/4sec/Wav2vec/Wav2vec_eval2.npz")
eval_Wav2vec = eval_Wav2vec['input_values']

X_test = eval_Wav2vec
y_test = eval["labels"][len(eval)//2:]

In [ ]:
import pandas as pd

# Assuming 'labels' is your Pandas Series
y_test.reset_index(drop=True, inplace=True)

In [ ]:
from sklearn.metrics import roc_curve
from scipy.optimize import brentq
from scipy.interpolate import interp1d

def stack_loader(X, y):
  X_tensor = torch.tensor(X, dtype=torch.float32)
  y_tensor = torch.tensor(y)
  dataset = TensorDataset(X_tensor, y_tensor)
  data_loader = DataLoader(dataset, batch_size=32)
  return data_loader

def evaluate_eer(dev_loader, model, device):
    model.eval()
    all_scores = []
    all_labels = []
    for batch_x, batch_y in dev_loader:
        batch_x = batch_x.to(device)
        batch_out = model(batch_x)
        scores = torch.softmax(batch_out, dim=1)[:, 1].cpu().detach().numpy()  # Get the probabilities for class 1
        labels = batch_y.cpu().detach().numpy()
        all_scores.extend(scores)
        all_labels.extend(labels)

    fpr, tpr, thresholds = roc_curve(all_labels, all_scores, pos_label=1)
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    return eer

test_loader = stack_loader(X_test, y_test)

test_accuracy = evaluate_accuracy(test_loader, model, device)
test_eer = evaluate_eer(test_loader, model, device)

In [ ]:
X_test.shape

(22976, 64000)

In [ ]:
y_test.shape

(22976,)

In [ ]:
print("eval accuracy: ", test_accuracy)
print("eval err: ", test_eer)

eval accuracy:  93.61942896935933
eval err:  0.11928602525056899


In [ ]:
# eval accuracy:  93.74129526462396
# eval err:  0.1156914893617022

# eval accuracy:  93.61942896935933
# eval err:  0.11928602525056899

In [ ]:
thai_df.labels.value_counts()

0    49891
1    10000
Name: labels, dtype: int64